In [1]:
import torch
from torchvision.models import mobilenet_v2
from torchvision.models.feature_extraction import create_feature_extractor


In [ ]:
model = mobilenet_v2(weights = 'MobileNet_V2_Weights.DEFAULT')
#model.eval()


'MobileNetV2(\n  (features): Sequential(\n    (0): Conv2dNormActivation(\n      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)\n      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n      (2): ReLU6(inplace=True)\n    )\n    (1): InvertedResidual(\n      (conv): Sequential(\n        (0): Conv2dNormActivation(\n          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)\n          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n          (2): ReLU6(inplace=True)\n        )\n        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)\n        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n      )\n    )\n    (2): InvertedResidual(\n      (conv): Sequential(\n        (0): Conv2dNormActivation(\n          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)\n          (1

In [ ]:
# Define Transforms
from matplotlib import transforms


train_transform = transforms.Compose([
    # Input is 96x96. We RandomCrop to 64x64
    transforms.RandomCrop(64),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    # CenterCrop to 64x64 for testing
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load Datasets
# Note: We load the training data twice to apply different transforms to train vs val subsets seamlessly
# or we can rely on Subset. However, Subset inherits the dataset's transform.
# A cleaner way is to create two dataset instances.

full_train_dataset = datasets.STL10(root='./data', split='train', download=True, transform=None)
test_dataset = datasets.STL10(root='./data', split='test', download=True, transform=test_transform)

# Split indices
targets = full_train_dataset.labels
train_idx, val_idx = train_test_split(
    np.arange(len(full_train_dataset)),
    test_size=0.2,
    shuffle=True,
    stratify=targets,
    random_state=42
)

# Create Subsets
# To properly handle transforms, we'll create two base datasets with correct transforms
train_base = datasets.STL10(root='./data', split='train', download=True, transform=train_transform)
val_base = datasets.STL10(root='./data', split='train', download=True, transform=test_transform)

# Full training datasets for final training
train_ds_aug = train_base
train_ds_clean = val_base

train_set = Subset(train_base, train_idx)
val_set = Subset(val_base, val_idx)

# Create DataLoader for test evaluation
batch_size = 128 # Increased batch size for more stable gradients
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Train Size: {len(train_set)} | Val Size: {len(val_set)} | Test Size: {len(test_dataset)}")